In [55]:
#Modelo 3
import numpy as np
import pandas as pd
import yfinance as yf
import statsmodels.api as sm
from scipy.optimize import curve_fit
import plotly.graph_objects as go

# 1. Descargar datos históricos del activo
activo = "AAPL"  # Ticker del activo
periodo = "1y"
datos = yf.Ticker(activo).history(period=periodo)

# 2. Calcular los retornos promedio (Average) y la desviación estándar (std)
datos['Average'] = datos[['High', 'Low']].mean(axis=1).pct_change()
datos['std'] = datos['Average'].rolling(window=5).std()
datos['log_std'] = np.log(datos['std'])
datos['log_Average'] = np.log(datos['Average'].replace(0, np.nan).dropna())

# Eliminar valores NaN para los ajustes
datos.dropna(inplace=True)

# --- MODELO LOGARITMICO ---
# Ajustar el modelo exponencial transformado con logaritmos
X_exp_log = sm.add_constant(datos['Average'])  # Log(σ) = Log(α) + β * Average
modelo_exp_log = sm.OLS(datos['log_std'], X_exp_log).fit()

# Coeficientes del modelo exponencial con logaritmos
beta_exp_log = modelo_exp_log.params[1]

# Predicciones del modelo exponencial con logaritmos
datos['log_std_pred_exp'] = modelo_exp_log.params[0]+ beta_exp_log * datos['Average']

# --- GRAFICAR LOS RESULTADOS ---
fig = go.Figure()

# Datos observados
fig.add_trace(go.Scatter(x=datos['Average'], y=datos['std'],
                         mode='markers', name='Datos Observados',
                         marker=dict(size=8, color='blue', opacity=0.7)))


# Definir x_vals como un rango de valores basado en la columna 'Average'
x_vals = np.linspace(datos['Average'].min(), datos['Average'].max(), 100)

# Línea del modelo exponencial
y_vals_exp = np.exp(modelo_exp_log.params[0] + beta_exp_log * x_vals)
fig.add_trace(go.Scatter(x=x_vals, y=y_vals_exp, mode='lines', name='Modelo Exponencial',
                         line=dict(color='green')))

# Configuración del gráfico
fig.update_layout(title="Modelos Polinomial y Exponencial | Desv. Estándar vs Retornos Promedio",
                  xaxis_title="Retornos Promedio (Average)",
                  yaxis_title="Desviación Estándar (std)",
                  template="plotly_white")

fig.show()

# --- RESULTADOS ---
print("Resultados del Modelo Polinomial:")
print(f"Intercepto (α): {alpha_poly_log:.4f}")
print(f"Coeficiente Lineal (β1): {beta_1_poly_log:.4f}")
print(f"Coeficiente Cuadrático (β2): {beta_2_poly_log:.4f}")
print(f"Resumen del Modelo Polinomial:\n{modelo_poly_log.summary()}")

print("\nResultados del Modelo Exponencial:")
print(f"Intercepto (α): {alpha_exp_log:.4f}")
print(f"Coeficiente Exponencial (β): {beta_exp_log:.4f}")


C:\Users\retir\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\pandas\core\arraylike.py:399: RuntimeWarning:

invalid value encountered in log

C:\Users\retir\AppData\Local\Temp\ipykernel_9748\3521186495.py:29: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

C:\Users\retir\AppData\Local\Temp\ipykernel_9748\3521186495.py:32: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

C:\Users\retir\AppData\Local\Temp\ipykernel_9748\3521186495.py:47: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys 

Resultados del Modelo Polinomial:
Intercepto (α): -4.9298
Coeficiente Lineal (β1): 1.4810
Coeficiente Cuadrático (β2): -0.3178
Resumen del Modelo Polinomial:
                            OLS Regression Results                            
Dep. Variable:                log_std   R-squared:                       0.145
Model:                            OLS   Adj. R-squared:                  0.132
Method:                 Least Squares   F-statistic:                     11.59
Date:                Thu, 30 Jan 2025   Prob (F-statistic):           2.24e-05
Time:                        20:12:56   Log-Likelihood:                -113.42
No. Observations:                 140   AIC:                             232.8
Df Residuals:                     137   BIC:                             241.7
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|

In [56]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Coeficientes del modelo exponencial con logaritmos
alpha_exp_log = np.exp(modelo_exp_log.params[0])
beta_exp_log = modelo_exp_log.params[1]

# Generar valores de x
x_vals = np.linspace(datos['Average'].min(), datos['Average'].max(), 500)

# Calcular valores de y para el modelo exponencial con logaritmos
std_exp_log_vals = alpha_exp_log * np.exp(beta_exp_log * x_vals)

# Filtrar valores dentro del rango [0, 0.011)
rango_mask_exp_log = (std_exp_log_vals >= 0) & (std_exp_log_vals < 0.0)

# Crear diccionario con los datos filtrados
datos_filtrados = {
    'Average': x_vals[rango_mask_exp_log],
    'Risk': std_exp_log_vals[rango_mask_exp_log]
}

# Convertir a DataFrame
df_filtrado = pd.DataFrame(datos_filtrados)

# Calcular máximos y minimos, acotación de la ecuación.
max_average = df_filtrado['Average'].max()
min_average = df_filtrado['Average'].min()
max_risk = df_filtrado['Risk'].max()
min_risk = df_filtrado['Risk'].min()

# Mostrar resultados
if np.isnan(max_average) and np.isnan(min_average) and np.isnan(max_risk) and np.isnan(min_risk):
    print(f"""No se puede recomendar esta acción dado que su riesgo nivel de riesgo es demasiado alto para poder procesar la relación retorno
           y riesgo, se recomienda tomar todas las precauciones necesarias al invertir, dado la posibilidad de pérdida total o parcial de la acción""")
else:
    print(f"Máximo de 'Average': {max_average:.6f}")
    print(f"Mínimo de 'Average': {min_average:.6f}")
    print(f"Máximo de 'Risk': {max_risk:.6f}")
    print(f"Mínimo de 'Risk': {min_risk:.6f}")


No se puede recomendar esta acción dado que su riesgo nivel de riesgo es demasiado alto para poder procesar la relación retorno
           y riesgo, se recomienda tomar todas las precauciones necesarias al invertir, dado la posibilidad de pérdida total o parcial de la acción


C:\Users\retir\AppData\Local\Temp\ipykernel_9748\4183658941.py:6: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

C:\Users\retir\AppData\Local\Temp\ipykernel_9748\4183658941.py:7: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`



In [53]:
fig = go.Figure()

# Datos observados
fig.add_trace(go.Scatter(x=datos['Average'], y=datos['std'],
                         mode='markers', name='Datos Observados',
                         marker=dict(size=8, color='blue', opacity=0.7)))


# Definir x_vals como un rango de valores basado en la columna 'Average'
x_vals = np.linspace(datos['Average'].min(), datos['Average'].max(), 100)

# Línea del modelo exponencial
y_vals_exp = np.exp(alpha_exp_log * np.exp(beta_exp_log * x_vals))
fig.add_trace(go.Scatter(x=x_vals, y=y_vals_exp, mode='lines', name='Modelo Exponencial',
                         line=dict(color='green')))

# Configuración del gráfico
fig.update_layout(title="Modelos Polinomial y Exponencial | Desv. Estándar vs Retornos Promedio",
                  xaxis_title="Retornos Promedio (Average)",
                  yaxis_title="Desviación Estándar (std)",
                  template="plotly_white")

fig.show()

print("Resultados del Modelo Polinomial:")
print(f"Intercepto (α): {alpha_poly_log:.4f}")
print(f"Coeficiente Lineal (β1): {beta_1_poly_log:.4f}")
print(f"Coeficiente Cuadrático (β2): {beta_2_poly_log:.4f}")
print(f"Resumen del Modelo Polinomial:\n{modelo_poly_log.summary()}")

print("\nResultados del Modelo Exponencial:")
print(f"Intercepto (α): {alpha_exp_log:.4f}")
print(f"Coeficiente Exponencial (β): {beta_exp_log:.4f}")

Resultados del Modelo Polinomial:
Intercepto (α): -4.9298
Coeficiente Lineal (β1): 1.4810
Coeficiente Cuadrático (β2): -0.3178
Resumen del Modelo Polinomial:
                            OLS Regression Results                            
Dep. Variable:                log_std   R-squared:                       0.145
Model:                            OLS   Adj. R-squared:                  0.132
Method:                 Least Squares   F-statistic:                     11.59
Date:                Thu, 30 Jan 2025   Prob (F-statistic):           2.24e-05
Time:                        20:12:10   Log-Likelihood:                -113.42
No. Observations:                 140   AIC:                             232.8
Df Residuals:                     137   BIC:                             241.7
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|